In [1]:
import sys
sys.executable

'/mnt/c/users/nawfal/documents/apps/collaborative project/cctv_motion_detection/motion_venv/bin/python'

In [2]:
import cv2
import os
import time
import numpy as np
import csv

# Camera Check

In [3]:
# cap = cv2.VideoCapture("http://192.168.50.234:5000/video")
cap = cv2.VideoCapture("http://192.168.100.197:5000/video")


In [4]:
# Initialize the camera

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

# Capture frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Display the frame
    cv2.namedWindow("Camera Capture", cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow("Camera Capture", 640, 480)
    cv2.imshow("Camera Capture", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


# Import Mediapipe

In [5]:
import cv2
import os
import time
import csv
import warnings
import pickle

import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


2025-09-02 09:23:09.689425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756776189.936104     860 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756776190.008722     860 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756776190.615626     860 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756776190.615658     860 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756776190.615660     860 computation_placer.cc:177] computation placer alr

# Z Coordinate detection
Zoom in & Zoom Out while move forward and backwards

In [28]:
def check_nose_z(lndmrk_z):
    nose = lndmrk_z["nose"]
    ear_l = lndmrk_z["ear_l"]
    ear_r = lndmrk_z["ear_r"]

    if nose < (ear_l and ear_r):
        return "Hadap Depan"
    else:
        return "Hadap Belakang"

    return f"nose: {nose:.5f}, ear: {ear_l:.5f}"

In [30]:
lndmrkX = None
lndmrkY = None
lndmrkZ = None

cap = cv2.VideoCapture("http://192.168.100.197:5000/video")


time.sleep(3)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get specific landmarks
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
        ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]

        show_landmark_list = landmark_pb2.NormalizedLandmarkList()
        show_landmark_list.landmark.extend([nose])

        nose_z = check_nose_z({"nose": nose.z, "ear_l": ear_l.z, "ear_r": ear_r.z})

        # Draw landmarks
        for lndmrk_z in show_landmark_list.landmark:
            x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
            lndmrkX = lndmrk_z.x
            lndmrkY = lndmrk_z.y
            lndmrkZ = lndmrk_z.z

        cv2.putText(
            image,
            f"x: ({nose_z})",
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
            cv2.LINE_AA
        )
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1756793829.961641     860 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756793829.991000   54112 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1756793830.388888   54100 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756793830.627939   54101 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756793830.671005   54105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756793830.714023   54109 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

move forward goes till minus

# Face Direction Detection

In [ ]:
def check_direction(landmark):
    nose = landmark["nose"]
    ear_r = landmark["ear_r"]
    ear_l = landmark["ear_l"]

    # Calculate distances
    dist_left = abs(nose - ear_l)
    dist_right = abs(nose - ear_r)
    ear_gap = abs(ear_r - ear_l)

    # Thresholds (tune based on your camera and position)
    center_threshold = 0.05    # nose roughly centered
    side_threshold = 0.02      # nose very close to one ear (90-degree)
    min_ear_gap = 0.05         # ears almost overlapped in X axis at side profile

    # Detect 90-degree turns first
    if ear_gap < min_ear_gap:
        if dist_right < dist_left:
            return "90-right"
        else:
            return "90-left"

    # Detect regular left/right/center
    if dist_left - dist_right > center_threshold:
        return "left"
    elif dist_right - dist_left > center_threshold:
        return "right"
    else:
        return "center"


# Camera Check

In [ ]:
camera = cv2.VideoCapture("http://192.168.100.197:5000/video")

# Check if the camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic :
    
    while camera.isOpened():  
        ret, frame = camera.read()

        if not ret:
            print("❌ Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # =============== THE LANDMARKS ============== #
        
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
        ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
        wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

        # Array Each Landmark
        face_direction = check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}) 

        # For Draw
        lm = landmark_pb2.NormalizedLandmarkList()
        lm.landmark.extend([nose, ear_r, ear_l, wrist_r])

        # ============================================ #

        for lndmrk_z in lm.landmark:
            x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
            cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
        
            cv2.putText(
                image,
                f"x: ({face_direction})",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )
        # =============== DETECTION ================== #
        # ============================================ #

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


I0000 00:00:1756785321.547105     860 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756785321.597987   28259 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1756785322.221218   28249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756785322.316595   28257 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756785322.331171   28248 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756785322.331452   28249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

Left & Right Eye Can Be Used For the Left and Right

# Body Rotation State

In [ ]:
camera = cv2.VideoCapture("http://192.168.100.197:5000/video")

# Check if the camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic :
    
    state = ""

    while camera.isOpened():  
        ret, frame = camera.read()

        if not ret:
            print("❌ Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # =============== THE LANDMARKS ============== #
        
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
        ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
        wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

        # Array Each Landmark
        # face_direction = check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}) 

        # For Draw
        lm = landmark_pb2.NormalizedLandmarkList()
        lm.landmark.extend([nose, ear_r, ear_l, wrist_r])

        # ============================================ #

        for lndmrk_z in lm.landmark:
            x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
            cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
        
            # cv2.putText(
            #     image,
            #     f"x: ({face_direction})",
            #     (50, 50),
            #     cv2.FONT_HERSHEY_SIMPLEX,
            #     0.7,
            #     (0, 255, 0),
            #     2,
            #     cv2.LINE_AA
            # )
        # =============== DETECTION ================== #
        # ============================================ #

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()